# Are there two (non-isomorphic) graphs with n vertices which have a winning classical strategy?

Two graphs G, H

Three players: A, B, C

I = O = V(G) = V(H)

The three players win a round if

(1) output for each player is from the other graph than the input

(2) if the inputs x_A, x_B, x_C are from G then rel(x_A, x_B, x_C) = rel(y_A, y_B, y_C)

i.e. x_A = x_B = x_C iff y_A = y_B = y_C

x_A ~ x_B ~ x_C ~ x_A iff y_A ~ y_B ~ y_C ~ y_A (triangles)

x_A !~ x_B !~ x_C !~ x_A iff y_A !~ y_B !~ y_C !~ y_A (anti-triangles)

(3) If (WLOG) x_A, x_B are from G and x_C is from H, then

x_A = y_C iff y_A = x_C

x_B = y_C iff y_B = x_C


In [1]:
numvertices = 4

#how many free entries in an adjacency matrix
numentries = int(numvertices*(numvertices-1)/2)
numgraphs = 2**numentries

print('We are considering ',numvertices, 
      ' vertices. This gives a total of ', 
      numgraphs, ' adjacency matrices.')

We are considering  4  vertices. This gives a total of  64  adjacency matrices.


In [2]:
#this is a weird conversion, see notes
#basically, it maps numentries to row/column indices
def WhereInMatrix(numvertices,k):
    rowindex = 0
    startrow = 0
    rowlength = numvertices - 1
    while k >= startrow + rowlength:
        startrow = startrow + rowlength
        rowindex += 1
        rowlength -= 1
    colindex = rowindex + 1 + (k - startrow)
    return [rowindex, colindex]

In [4]:
import itertools
import numpy
import copy

# find all possible subsets of [0,1,...,numentries]
# to determine which are 0 and which are 1
combos = []
for k in range(numentries+1):
    which = list(itertools.combinations(range(numentries), k))
    for i in range(len(which)):
        combos.append(which[i])
print('Number of adjacency matrices = len(combos):', len(combos))
#print('combos:', combos)

#initialize list of all distinct adjacency matrices
AllAdj = []

#go from combos to Adjacency matrix
for index in range(len(combos)): #for each combination
    #initialize adjacency matrix
    Adj = [[0]*numvertices for _ in range(numvertices)]
    for entry in range(len(combos[index])):
        #add 1's in appropriate spots
        [rowindex,colindex] = WhereInMatrix(numvertices,combos[index][entry])
        Adj[rowindex][colindex] = 1
        Adj[colindex][rowindex] = 1
    AllAdj.append(Adj)

#print('AllAdj:', AllAdj)
#now we have all adjacency matrices as a big jumbo list!

Number of adjacency matrices = len(combos): 64


In [5]:
def IsoGraphs(matrix1,matrix2,perm):
    """
    Suppose we permute the indices of matrix2 by perm
    Check if matrix1 = matrix2 permuted
    """
    n = len(matrix1)
    if len(matrix2) != n:
        return False
    
    newmatrix2 = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            newmatrix2[i][j] = matrix2[perm[i]][perm[j]]
            if newmatrix2[i][j] != matrix1[i][j]:
                return False
    if matrix1 == newmatrix2:
        return True
    else:
        return False


In [6]:
# compare 2 matrices to see if they're classically isomorphic

#create list of vertices
vertices = []
for i in range(numvertices):
    vertices.append(i)
print('vertices:',vertices)

# Get all permutations of [1, 2, 3,..., numvertices] 
permutations = itertools.permutations(vertices) 
permlist = list(permutations)
#print('permlist:',permlist)

matrix1 = 0
matrix2 = matrix1 + 1
while matrix1 < len(AllAdj):
    while matrix2 < len(AllAdj):
        #print('comparing matrix1:',matrix1,' and matrix2:',matrix2)
        matrix1iso2 = False
        p = 0
        while matrix1iso2 == False and p < len(permlist):
            #print('compare matrix1 with matrix2 edited by perm', permlist[p])
            matrix1iso2 = IsoGraphs(AllAdj[matrix1],AllAdj[matrix2],permlist[p])
            p += 1
        
        if matrix1iso2 == True:
            #print('they are isomorphic, delete matrix', matrix2)
            AllAdj.pop(matrix2) #delete matrix2 from list
        else:
            #print('they are not isomorphic')
            matrix2 += 1
    
    matrix1 += 1
    matrix2 = matrix1 + 1


print('\n\nFor ', numvertices, 
      ' vertices, there are ', len(AllAdj), ' non-isomorphic graphs.')
    

vertices: [0, 1, 2, 3]


For  4  vertices, there are  11  non-isomorphic graphs.


In [7]:
def WinningClassical(matrix1,matrix2,vertices,perm):
    """
    Suppose we permute the indices of matrix2 by perm
    Check if matrix1 and newmatrix2 have a winning classical
    multiplayer strategy
    Here, we assume vertex i in matrix1 maps to vertex i
    in newmatrix2
    """
    n = len(matrix1)
    dummy = 0
    WinningClassicalStrategy = True
    
    isomorphic = IsoGraphs(matrix1,matrix2,perm)
    if isomorphic == True:
        return False
    else:
        newmatrix2 = [[0]*n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                newmatrix2[i][j] = matrix2[perm[i]][perm[j]]

        for i, j, k in itertools.combinations(vertices, 3):
            if matrix1[i][j] == 1 and matrix1[i][k] == 1 and matrix1[j][k] == 1:
                if newmatrix2[i][j] == 1 and newmatrix2[i][k] == 1 and newmatrix2[j][k] == 1:
                    dummy = 1
                else:
                    WinningClassicalStrategy = False
            if matrix1[i][j] == 0 and matrix1[i][k] == 0 and matrix1[j][k] == 0:
                if newmatrix2[i][j] == 0 and newmatrix2[i][k] == 0 and newmatrix2[j][k] == 0:
                    dummy = 1
                else:
                    WinningClassicalStrategy = False
            if newmatrix2[i][j] == 1 and newmatrix2[i][k] == 1 and newmatrix2[j][k] == 1:
                if matrix1[i][j] == 1 and matrix1[i][k] == 1 and matrix1[j][k] == 1:
                    dummy = 1
                else:
                    WinningClassicalStrategy = False
            if newmatrix2[i][j] == 0 and newmatrix2[i][k] == 0 and newmatrix2[j][k] == 0:
                if matrix1[i][j] == 0 and matrix1[i][k] == 0 and matrix1[j][k] == 0:
                    dummy = 1
                else:
                    WinningClassicalStrategy = False

        if WinningClassicalStrategy == True:
            print('the following two graphs are not isomorphic but do have a winning classical strategy')
            print(matrix1)
            print(newmatrix2)
        return WinningClassicalStrategy



In [7]:
"""
compare 2 matrices to see if there's a winnning classical strategy
"""

#create list of vertices
vertices = []
for i in range(numvertices):
    vertices.append(i)
print('vertices:',vertices)

# Get all permutations of [1, 2, 3,...,numvertices] 
permutations = itertools.permutations(vertices) 
permlist = list(permutations)

numpairs = 0
matrix1 = 0
matrix2 = matrix1 + 1
while matrix1 < len(AllAdj):
    while matrix2 < len(AllAdj):
        #print('comparing matrix1:',matrix1,' and matrix2:',matrix2)
        matrix1iso2 = False
        p = 0
        while matrix1iso2 == False and p < len(permlist):
            #print('compare matrix1 with matrix2 edited by perm', permlist[p])
            matrix1iso2 = WinningClassical(AllAdj[matrix1],AllAdj[matrix2],vertices,permlist[p])
            p += 1
        if matrix1iso2 == True:
            numpairs += 1
        
        matrix2 += 1
    
    matrix1 += 1
    matrix2 = matrix1 + 1

print('the number of pairs of graphs that have a winning classical multiplayer strategy is', 
     numpairs)

vertices: [0, 1, 2, 3]
the following two graphs are not isomorphic and have a winning classical strategy
[[0, 1, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
[[0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]
the following two graphs are not isomorphic and have a winning classical strategy
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]
[[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]]
the following two graphs are not isomorphic and have a winning classical strategy
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]
[[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 1], [0, 1, 1, 0]]
the following two graphs are not isomorphic and have a winning classical strategy
[[0, 1, 1, 0], [1, 0, 1, 0], [1, 1, 0, 0], [0, 0, 0, 0]]
[[0, 1, 1, 1], [1, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 0]]
the following two graphs are not isomorphic and have a winning classical strategy
[[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]]
[[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 1], [0, 1, 1, 0